# Imports

In [ ]:
import numpy as np
import scipy.linalg as spla
import scipy.sparse as sps
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.grid'] = True

# Penzl's FOM model

The model is described in [MOR Wiki](https://morwiki.mpi-magdeburg.mpg.de/morwiki/index.php/FOM).

In [ ]:
from pymor.models.iosys import LTIModel

In [ ]:
n = 100
A1 = np.array([[-1, 100], [-100, -1]])
A2 = np.array([[-1, 200], [-200, -1]])
A3 = np.array([[-1, 400], [-400, -1]])
A4 = sps.diags(np.arange(-1, -n - 1, -1))
A = sps.block_diag([A1, A2, A3, A4], format='csc')
B = np.ones((n + 6, 1))
B[:6, 0] = 10
C = B.T.copy()

In [ ]:
fom = LTIModel.from_matrices(A, B, C)

# Bode plot

Draw the Bode and/or magnitude plot

In [ ]:
w = np.logspace(-1, 6, 1000)
_ = fom.mag_plot(w)

# Approximation at $\infty$

Perform a Galerkin projection using $\mathcal{K}_r(A, b)$. Compute the basis matrix using the [Arnoldi iteration](https://en.wikipedia.org/wiki/Arnoldi_iteration#The_Arnoldi_iteration). You will need the following:

- `fom.A.source.empty()` to generate the initial empty basis
- `fom.B.as_range_array()` to transform the $B$ operator into a `VectorArray`
- `norm` method of `VectorArrays`
- `scal` method of `VectorArrays`
- `append` method of `VectorArrays`
- `fom.A.apply` to apply the $A$ operator
- `inner` method of `VectorArrays`
- `axpy` method of `VectorArrays`

In [ ]:
from pymor.reductors.basic import LTIPGReductor

In [ ]:
r = 10
V_inf = fom.A.source.empty()
v = fom.B.as_range_array()
v.scal(1 / v.norm()[0])
V_inf.append(v)
for i in range(1, r):
    v = fom.A.apply(V_inf[-1])
    for j in range(i):
        v.axpy(-V_inf[j].inner(v)[0, 0], V_inf[j])
    v.scal(1 / v.norm()[0])
    V_inf.append(v)

In [ ]:
pg_inf = LTIPGReductor(fom, V_inf, V_inf)

In [ ]:
rom_pg_inf = pg_inf.reduce()

In [ ]:
err_pg_inf = fom - rom_pg_inf

In [ ]:
fig, ax = plt.subplots()
fom.mag_plot(w, ax=ax)
_ = rom_pg_inf.mag_plot(w, ax=ax, linestyle='--')

In [ ]:
_ = err_pg_inf.mag_plot(w)

# Approximation at $0$

Perform a Galerkin projection using $\mathcal{K}_r(A^{-1}, A^{-1} b)$. As above, use the Arnoldi iteration to generate the basis matrix. You will need to use the `fom.A.apply_inverse` method to solve linear systems involving the $A$ operator.

In [ ]:
r = 10
V_zero = fom.A.source.empty()
v = fom.A.apply_inverse(fom.B.as_range_array())
v.scal(1 / v.norm()[0])
V_zero.append(v)
for i in range(1, r):
    v = fom.A.apply_inverse(V_zero[-1])
    for j in range(i):
        v.axpy(-V_zero[j].inner(v)[0, 0], V_zero[j])
    v.scal(1 / v.norm()[0])
    V_zero.append(v)

In [ ]:
pg_zero = LTIPGReductor(fom, V_zero, V_zero)

In [ ]:
rom_pg_zero = pg_zero.reduce()

In [ ]:
err_pg_zero = fom - rom_pg_zero

In [ ]:
fig, ax = plt.subplots()
fom.mag_plot(w, ax=ax)
_ = rom_pg_zero.mag_plot(w, ax=ax, linestyle='--')

In [ ]:
_ = err_pg_zero.mag_plot(w)

# Approximate at the peaks

Perform a Galerkin projection to approximate near the peaks ($100 i$, $200 i$, $400 i$) or use the `LTIBHIReductor` to do bitangential Hermite interpolation.

In [ ]:
from pymor.reductors.interpolation import LTIBHIReductor

In [ ]:
sigma = [100j, 200j, 400j, -100j, -200j, -400j]
b = fom.B.source.ones(6)
c = fom.B.source.ones(6)

In [ ]:
bhi = LTIBHIReductor(fom)
rom_bhi = bhi.reduce(sigma, b, c)

In [ ]:
err_bhi = fom - rom_bhi

In [ ]:
fig, ax = plt.subplots()
fom.mag_plot(w, ax=ax)
_ = rom_bhi.mag_plot(w, ax=ax, linestyle='--')

In [ ]:
_ = err_bhi.mag_plot(w)